In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re 

In [2]:
s = Service(r'C:\Users\natas\AppData\Local\Chrome\chromedriver.exe')
#AppData\Local\Chrome\chromedriver.exe')
driver = webdriver.Chrome(r'C:\Users\natas\AppData\Local\Chrome\chromedriver.exe')

In [3]:
def get_by_id(id_punk):
    driver.get("https://www.larvalabs.com/cryptopunks/details/"+str(id_punk))
    content=driver.page_source.encode('utf-8').strip()
    soup= BeautifulSoup(content,"html.parser")
    return soup

In [4]:
#type
def get_type(soup):
    type_punk=soup.findAll("div",{"class": "col-md-10"})
    return type_punk[0].find("a").text

In [5]:
#attributts
def get_attributts(soup):
    list_attr=[]
    attr=soup.findAll("div",{"class": "col-md-4"})
    for i in range(1,len(attr)):
        attr_text=attr[i].find("a").text
        list_attr.append(attr_text)
    print(list_attr)
    return list_attr

In [36]:
#regular expressions
def get_value(pattern, line):
    value=""
    match = re.search(pattern,line) 
    if match:
        value=match[0]
    return value    

In [13]:
_id=365
#dict_punk={}
soup_punk=get_by_id(_id)
attr=get_attributts(soup_punk)
types=get_type(soup_punk)
dict_punk[_id]={"type":types,"attr":attr}
dict_punk

['Normal Beard', 'Do-rag']


{12: {'type': 'Female', 'attr': ['Blonde Short', 'Purple Lipstick']},
 120: {'type': 'Female', 'attr': ['Tassle Hat', 'Big Shades']},
 365: {'type': 'Male', 'attr': ['Normal Beard', 'Do-rag']}}

In [ ]:
list_attr=[]
for k in range(0,10):
    soup=get_by_id(k)
    attr=soup.findAll("div",{"class": "col-md-4"})
    for i in range(1,len(attr)):
        attr_text=attr[i].find("a").text
        print(attr_text)
        list_attr.append(attr_text)
    return list_attr

In [15]:
#transaction
transaction_rows = soup_punk.find_all('tr')
#transaction_rows


In [17]:
tab_data = [[cell.text for cell in row.find_all(['th', 'td'])] for row in transaction_rows]
df = pd.DataFrame(tab_data)
header = df.iloc[0] 
df = df[1:]
df.columns = header
df.head(30)

,Type,From,To,Amount,Txn
1,Offer Withdrawn,,,,"Sep 01, 2021"
2,Offered,,,400Ξ ($1.16M),"Aug 06, 2021"
3,Sold,0x0d82eb,bensharf…,"39.89Ξ ($107,304)","Aug 05, 2021"
4,Offered,,,"39.89Ξ ($88,581)","Jun 29, 2021"
5,Sold,0x577ebc,0x0d82eb,"17.99Ξ ($73,442)","May 14, 2021"
6,Bid,0x0d82eb,,"17.99Ξ ($73,442)","May 14, 2021"
7,Offered,,,"19.99Ξ ($79,455)","May 11, 2021"
8,Sold,quantbik…,0x577ebc,"16.66Ξ ($66,219)","May 11, 2021"
9,Offered,,0x577ebc,"16.66Ξ ($67,336)","May 11, 2021"
10,(Unwrap),WrappedC…,quantbik…,,"May 11, 2021"


In [59]:
def get_user(str_user):
    pattern = "[=]\w+"
    if str_user!=None:
        user = get_value(pattern,to['href'])[1:]        
    else:
        user=""
    return user

row_info = []
pattern_amount ="[$][^)]+"
for element in transaction_rows:
    tr=element.find_all(['th', 'td'])
    _type_tr=tr[0].text
    _from=get_user(tr[1].find("a",href=True))
    _to=get_user(tr[2].find("a",href=True))
    amount=get_value(pattern_amount,tr[3].text)[1:]
    _date=tr[4].text
    print(_type_tr, "  ",_from, "  ", _to, "  ", amount, "  ", _date)


Type                Txn
Offer Withdrawn                Sep 01, 2021
Offered            1.16M    Aug 06, 2021
Sold     0xc352b534e8b987e036a93539fd6897f53488e56a    0xc352b534e8b987e036a93539fd6897f53488e56a    107,304    Aug 05, 2021
Offered            88,581    Jun 29, 2021
Sold     0xc352b534e8b987e036a93539fd6897f53488e56a    0xc352b534e8b987e036a93539fd6897f53488e56a    73,442    May 14, 2021
Bid     0xc352b534e8b987e036a93539fd6897f53488e56a        73,442    May 14, 2021
Offered            79,455    May 11, 2021
Sold     0xc352b534e8b987e036a93539fd6897f53488e56a    0xc352b534e8b987e036a93539fd6897f53488e56a    66,219    May 11, 2021
Offered        0xc352b534e8b987e036a93539fd6897f53488e56a    67,336    May 11, 2021
(Unwrap)    0xc352b534e8b987e036a93539fd6897f53488e56a    0xc352b534e8b987e036a93539fd6897f53488e56a        May 11, 2021
(Wrap)    0xc352b534e8b987e036a93539fd6897f53488e56a    0xc352b534e8b987e036a93539fd6897f53488e56a        May 04, 2021
Transfer    0xc352b534e8b987e